In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
df = pd.read_csv('household_power_consumption_days.csv')

In [14]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')

C:\Users\user\AppData\Local\Temp\ipykernel_15980\1276112075.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['datetime'] = pd.to_datetime(df['datetime'])


In [15]:
# Exclude non-numeric columns from scaling
numeric_columns = df.select_dtypes(include=[np.number]).columns
df_numeric = df[numeric_columns]

# Normalize the data
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)


In [16]:
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length].values
        label = data.iloc[i+seq_length].values
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [17]:
# Choose a sequence length (number of time steps to look back)
sequence_length = 10

In [20]:
X, y = create_sequences(df_scaled, sequence_length)

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [22]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=df_scaled.shape[1]))  # Output layer with the same number of features

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='mse')  # Use mean squared error for regression problems

In [24]:
# Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [25]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/50
33/33 [==============================] - 7s 31ms/step - loss: 0.0778 - val_loss: 0.0183
Epoch 2/50
33/33 [==============================] - 0s 12ms/step - loss: 0.0171 - val_loss: 0.0154
Epoch 3/50
33/33 [==============================] - 0s 12ms/step - loss: 0.0150 - val_loss: 0.0153
Epoch 4/50
33/33 [==============================] - 1s 16ms/step - loss: 0.0142 - val_loss: 0.0145
Epoch 5/50
33/33 [==============================] - 0s 14ms/step - loss: 0.0138 - val_loss: 0.0141
Epoch 6/50
33/33 [==============================] - 0s 12ms/step - loss: 0.0136 - val_loss: 0.0143
Epoch 7/50
33/33 [==============================] - 1s 17ms/step - loss: 0.0134 - val_loss: 0.0136
Epoch 8/50
33/33 [==============================] - 0s 11ms/step - loss: 0.0134 - val_loss: 0.0139
Epoch 9/50
33/33 [==============================] - 0s 11ms/step - loss: 0.0132 - val_loss: 0.0139
Epoch 10/50
33/33 [==============================] - 0s 14ms/step - loss: 0.0131 - val_loss: 0.0140


In [26]:
# Evaluate the model
mse = model.evaluate(X_test, y_test)
print(f'Test Mean Squared Error: {mse}')

9/9 [==============================] - 0s 6ms/step - loss: 0.0108
Test Mean Squared Error: 0.01077329833060503


In [27]:
# Make predictions on new data
# You can use the trained model to predict equipment failures on unseen data
predictions = model.predict(X_test)
# Implement thresholding based on your requirements


9/9 [==============================] - 0s 6ms/step


In [28]:
# Save the model if needed
model.save('predictive_maintenance_model.h5')

C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
